In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(ggpubr)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
currencies <- Currencies()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
startDate <- as.Date('2019-01-01')
currPair <- 'USDINR'
closestExpiries <- currencies$NseFuturesTimeSeries() %>%
    filter(TIME_STAMP >= startDate & SYMBOL == currPair) %>%
    group_by(TIME_STAMP) %>%
    summarize(EXPIRY_DT = min(EXPIRY)) 

nearestTimeSeries <- closestExpiries %>% 
    inner_join(currencies$NseFuturesTimeSeries(), by=c('TIME_STAMP' = 'TIME_STAMP', 'EXPIRY_DT' = 'EXPIRY')) %>%
    filter(TIME_STAMP >= startDate & SYMBOL == currPair) %>%
    select(TIME_STAMP, EXPIRY_DT, CLOSE, SETTLE) %>%
    arrange(TIME_STAMP)

nearestTimeSeries %>% print(n=100)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:     lazy query [?? x 4]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: TIME_STAMP
    TIME_STAMP EXPIRY_DT  CLOSE SETTLE
    <date>     <date>     <dbl>  <dbl>
  1 2019-01-01 2019-01-29  69.7   69.7
  2 2019-01-02 2019-01-29  70.4   70.4
  3 2019-01-03 2019-01-29  70.3   70.3
  4 2019-01-04 2019-01-29  69.9   69.9
  5 2019-01-07 2019-01-29  69.8   69.8
  6 2019-01-08 2019-01-29  70.4   70.4
  7 2019-01-09 2019-01-29  70.6   70.6
  8 2019-01-10 2019-01-29  70.5   70.5
  9 2019-01-11 2019-01-29  70.6   70.6
 10 2019-01-14 2019-01-29  71.0   71.0
 11 2019-01-15 2019-01-29  71.1   71.1
 12 2019-01-16 2019-01-29  71.3   71.3
 13 2019-01-17 2019-01-29  71.1   71.1
 14 2019-01-18 2019-01-29  71.2   71.2
 15 2019-01-21 2019-01-29  71.3   71.3
 16 2019-01-22 2019-01-29  71.5   71.5
 17 2019-01-23 2019-01-29  71.4   71.4
 18 2019-01-24 2019-01-29  71.1   71.1
 19 2019-01-25 2019-01-29  71.2   71.2
 20 2019-01-28 2019-01-29  71.1   71.1
 21 2019-01-29 2

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)